## System test

Let's check if the current user has admin rights or included into the 'video' group

In [1]:
!id

uid=1000(jovyan) gid=44(video) groups=44(video)


List video devices available for Jupyter

In [2]:
ls /dev/video*

/dev/video0   /dev/video14  /dev/video17  /dev/video20  /dev/videosync
/dev/video10  /dev/video15  /dev/video18  /dev/video21
/dev/video13  /dev/video16  /dev/video19  /dev/video23


## Video test

To read camera image the next algorithm can be used:

```python
# Activate camera object
cap = cv2.VideoCapture(0)

# Take a picture
_, frame = cap.read()

# Convert the picture to bytes
_, frame = cv2.imencode('.png', frame)
frame = frame.tobytes()```


In [1]:
import solara, cv2
from time import sleep

@solara.component
def Page(): 
    frame_react = solara.use_reactive(None)
    video_running_react = solara.use_reactive(False)
    cap_react = solara.use_reactive(None)
    
    def change_state():
        if not video_running_react.value:
            cap_react.value = cv2.VideoCapture(0)
            video_running_react.value = True
        else:
            video_running_react.value = False
            sleep(1)
            frame_react.value = None
            cap_react.value.release()
            
    def show_image():
        while True:
            if video_running_react.value and cap_react.value is not None:
                frame = cv2.flip(cap_react.value.read()[1], 1)
                frame_react.value = cv2.imencode('.png', frame)[1].tobytes()
    
    with solara.Card(title="Robot video"):
        solara.Button(
            label = 'Stop Video' if video_running_react.value else 'Start Video', 
                on_click = change_state)
        if frame_react.value is not None:
            solara.Image(frame_react.value) 

    solara.use_thread(show_image)

Page()

Cannot show ipywidgets in text